<a href="https://colab.research.google.com/github/estevaorev/Agente-Coach-Espiritual-Pessoal-git/blob/main/Imers%C3%A3o_IA_Alura_%2B_Google_Gemini_CoachAI_Espiritual.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip -q install google-genai

In [2]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [3]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [4]:
# Instalar Framework ADK de agentes do Google ################################################
%pip install -q google-adk


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.

In [5]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [6]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [7]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [8]:
##########################################
# --- Agente 1: Identificador de humor/necessidade --- #
##########################################
def agente_identificador(necessidade):
    identificador = Agent(
        name="agente_identificador",
        model="gemini-2.0-flash",

        instruction=f"""
            Propósito e Objetivos:
            * Identificar o sentimento, humor e necessidade do usuário com base em uma única frase fornecida.
            * Classificar a necessidade do usuário em uma das três categorias: Buscando Conforto, Inspiração ou Perspectiva.

            Comportamentos e Regras:

            1) Análise da Frase do Usuário:
            a) Analisar a frase fornecida pelo usuário para identificar palavras-chave, tom e contexto emocional.
            b) Inferir o estado emocional (sentimento e humor) do usuário com base na análise da frase.
            c) Determinar a necessidade principal do usuário (Conforto, Inspiração ou Perspectiva) com base no estado emocional inferido e no contexto da frase.

            2) Respostas Específicas por Categoria:

            a) Buscando Conforto:
            - Reconhecer e validar o sentimento de tristeza ou desconforto expresso pelo usuário.
            - Responder que está buscando Conforto.

            b) Buscando Inspiração:
            - Reconhecer o desejo do usuário por motivação ou um início positivo.
            - Responder que está buscando Inspiração.

            c) Buscando Perspectiva:
            - Reconhecer a situação de incerteza, dificuldade ou necessidade de clareza do usuário.
            - Responder que está buscando Perspectiva.

            3) Tom e Estilo:

            Exemplo de Interação (incorporado nas regras para clareza):
            * Usuário (Buscando Conforto): 'Hoje estou me sentindo bem triste.'
            * Chatbot: Reconhece a tristeza. Responde 'Buscando Conforto'.

            * Usuário (Buscando Inspiração): 'Quero uma mensagem para começar bem o dia.'
            * Chatbot: Reconhece a busca por inspiração matinal. Responde 'Buscando Inspiração'.

            * Usuário (Buscando Perspectiva): 'Estou com uma decisão difícil pela frente.'
            * Chatbot: Reconhece a incerteza/dificuldade. Responde 'Buscando Perspectiva'.""",

        description="Agente que identifica o sentimento, humor ou necessidade"

    )
    entrada_do_agente_identificador = necessidade
    # Executa o agente
    jornada = call_agent(identificador, entrada_do_agente_identificador)
    return jornada

In [21]:
# ... (definição da função agente_identificador) ...

##########################################
# --- Agente 2: Gerador de mensagem --- #
##########################################
def agente_motivador(jornada, preferencia_tom="acolhedor(a)"):
    instruction_motivador = f"""
Propósito e Objetivos:
* Gerar uma mensagem motivacional personalizada para o usuário com base na sua necessidade identificada.
* Oferecer conforto, inspiração ou perspectiva, dependendo da categoria da necessidade do usuário.
* Use um tom {preferencia_tom} na sua resposta.
* **Certifique-se de que a linguagem seja inclusiva e respeitosa com diversas visões espirituais e filosóficas, ou a ausência delas.** # <<-- ADICIONADO Ponto 4

Conteúdos espirituais/inspiracionais que você pode utilizar como referência (gere mensagens que se encaixem nestes estilos): versículos, citações, pequenos textos, parábolas, afirmações, trechos de livros.

Comportamentos e Regras:
1) Análise da Necessidade Identificada:
a) Receber a categoria da necessidade do usuário ({jornada}).
b) Entender o contexto da necessidade para personalizar a mensagem.

2) Respostas Específicas por Categoria:
a) Buscando Conforto:
- Reconhecer e validar o sentimento.
- Oferecer mensagem de empatia, apoio, esperança ou aceitação.
- Evitar conselhos não solicitados.
b) Buscando Inspiração:
- Reconhecer o desejo por motivação.
- Fornecer citação, reflexão ou afirmação.
- Manter a mensagem breve.
c) Buscando Perspectiva:
- Reconhecer incerteza/dificuldade.
- Oferecer mensagem sobre sabedoria, confiança ou resiliência.
- Evitar soluções diretas.

3) Tom e Estilo:
* Mantenha um tom empático, atencioso e respeitoso.
* Utilize linguagem clara e apropriada.
* Evite jargões complexos.
* Responda de forma direta.
* Lembre-se de seguir o tom {preferencia_tom} solicitado.
* **Garanta que as mensagens sejam inclusivas e respeitosas com todas as crenças ou não-crenças.** # <<-- REFORÇO Ponto 4

"""
    motivador = Agent(
        name="agente_motivador",
        model="gemini-2.5-flash-preview-04-17", # Use o modelo que você preferir
        instruction=instruction_motivador,
        description="Agente que fornece a resposta motivacional/espiritual."
    )

    entrada_do_agente_motivador = jornada
    mensagem = call_agent(
        motivador,
        entrada_do_agente_motivador
    )
    return mensagem


In [ ]:
# --- Loop externo para a sessão completa de conversa ---
while True:
    # --- DISCLAIMER E MENSAGEM DE PRIVACIDADE (Ponto 1 e 3) ---
    print("-" * 50)
    print("⚠️ Importante:")
    print("O CoachAI Espiritual não substitui terapia, aconselhamento psicológico ou atendimento médico profissional.")
    print("Em situações de sofrimento intenso ou crise, por favor, busque ajuda especializada imediatamente.")
    print("Suas interações são processadas de forma confidencial e não são armazenadas permanentemente nesta versão de demonstração.") # Adapte conforme sua política de privacidade real futura
    print("-" * 50)

    # --- Pergunta a preferência de Tom/Personalidade uma vez por sessão ---
    # Definir opções de tom/personalidade para o MVP
    opcoes_tom = {
        "amigo": "amigo(a) e acolhedor(a)",
        "sabio": "sábio(a) e reflexivo(a)",
        "direto": "direto(a) e conciso(a)"
    }
    preferencia_tom = ""
    # Loop para garantir que uma opção de tom válida seja escolhida
    while preferencia_tom not in opcoes_tom.values(): # Verificar se o valor está nas opções
        tom_input = input(f"""\n✨ Bem-vindo(a) ao CoachAI Espiritual! Que tipo de guia você prefere hoje?\n
        ({', '.join(opcoes_tom.keys())}, ou 'sair' para encerrar):\n """).strip().lower()

        if tom_input in ["sair", "encerrar", "parar", "quit"]:
            print("\nQue sua jornada seja iluminada! ✨\n") # Mantido seu ajuste de quebra de linha
            # Precisamos sair de ambos os loops se sair for digitado aqui
            exit_flag = True
            break # Quebra o loop de escolha de tom

        if tom_input in opcoes_tom:
            preferencia_tom = opcoes_tom[tom_input] # Armazena o valor completo (ex: "amigo(a) e acolhedor(a)")
            print(f"Certo! Seu guia será {preferencia_tom}.\n") # Mantido seu ajuste de quebra de linha
        else:
            print("Opção inválida. Por favor, escolha entre 'amigo', 'sabio', 'direto', ou 'sair'.\n") # Mantido seu ajuste de quebra de linha

    # Se a flag de sair foi definida no loop de escolha de tom, sair do loop externo também
    if 'exit_flag' in locals() and exit_flag:
        break # Quebra o loop externo

    # --- Loop interno para obter necessidade e interagir sobre ela ---
    # Este loop interno agora se inicia APÓS a definição do tom
    while True: # Loop para obter necessidade e interagir sobre ela
        # --- Obter entrada inicial ou nova do Usuário ---
        # Pergunta como ele está se sentindo ou qual a nova necessidade.
        # A opção de sair é tratada aqui e também na escolha inicial de tom.
        necessidade_input = input("""\n❓ Como você está se sentindo agora,\n
        ou qual sua necessidade \n
        (ou digite 'novo' para mudar o tom/personalidade ou 'sair' para encerrar)?\n """).strip() # Mantido seu ajuste de quebra de linha

        # Verificar se o usuário quer encerrar a conversa a partir daqui
        if necessidade_input.lower() in ["sair", "encerrar", "parar", "quit"]:
             print("\nQue sua jornada seja iluminada! ✨\n") # Mantido seu ajuste de quebra de linha
             sair_conversao_completa = True # Seta a flag para sair completamente
             break # Quebra o loop interno (necessidade)

        # Adicionar opção para MUDAR o tom/personalidade dentro da conversa
        if necessidade_input.lower() in ["novo", "nova frase", "mudar", "diferente"]:
             print("\nOkay, vamos refletir sobre o seu estado atual.\n") # Mantido seu ajuste de quebra de linha
             # Quebra o loop interno (necessidade) para voltar ao início do loop externo e escolher um novo tom.
             break # Quebra o loop interno (necessidade)


        # Verificar se a entrada está vazia (após remover espaços)
        if not necessidade_input:
            print("❌ Você precisa fornecer uma frase, digitar 'novo' para mudar o tom, ou 'sair' para encerrar.\n") # Mantido seu ajuste de quebra de linha
            continue # Volta para o início do loop interno (necessidade)

         # --- GERENCIAMENTO BÁSICO DE CRISE (Ponto 2) ---
        palavras_chave_crise = ["suicidio", "suicídio", "quero morrer", "me matar", "não aguento mais", "tirar a vida", "fim da linha"] # Lista de termos para detectar
        if any(palavra in necessidade_input.lower() for palavra in palavras_chave_crise):
            print("\nSua dor é real, e você não está sozinho(a). Existem pessoas que podem te ajudar agora mesmo.")
            print("Por favor, procure ajuda profissional imediata. Você pode ligar para o **CVV (Centro de Valorização da Vida) no número 188** a qualquer hora, gratuitamente e com sigilo.")
            print("Eles estão prontos para ouvir e oferecer apoio.")
            print("Você também pode buscar ajuda em um CAPS (Centro de Atenção Psicossocial) ou em um pronto-socorro próximo.")
            print("\nLembre-se, há esperança e ajuda disponível.")
            print("-" * 50)
            # Não chamar os agentes, apenas pedir nova entrada
            continue # Volta para o início do loop interno (necessidade) para pedir uma nova frase


        print("Aguarde! Estamos preparando sua mensagem: ") # Mantido seu print original

        # --- Usando agente 1 (Identificador de humor/necessidade) ---
        print("\n...🔍 Identificando sua necessidade...") # Mantido seu print original
        jornada_identificada_output = agente_identificador(necessidade_input)
        print("\n...📝 Resultado do agente 1 (Identificador) ---\n") # Mantido seu print original
        display(to_markdown(jornada_identificada_output))
        jornada_categoria = jornada_identificada_output.strip()

        # --- Usando agente 2 (Gerador de mensagem) ---
        print("📝 Gerando mensagem motivacional...\n") # Mantido seu print original
        # --- CHAVE DA PERSONALIZAÇÃO: Passar a preferência de tom para o agente ---
        # Chamada à função agente_motivador agora inclui a variável preferencia_tom
        mensagem = agente_motivador(jornada_categoria, preferencia_tom) # <<-- Mudança aqui!
        print("\n...📝 Resultado do agente 2 (Motivador) ---\n") # Mantido seu print original
        print("------------------------------------------------")
        display(to_markdown(mensagem))
        print("------------------------------------------------")
        print("\n✅ Mensagem generada com sucesso!\n") # Mantido seu print original
        print("------------------------------------------------")

        # --- Loop interno para opções PÓS-mensagem (já existente) ---
        # Este loop agora lida apenas com "outra mensagem para este tópico"
        # As opções "novo" e "sair" foram movidas para o loop de necessidade mais acima
        while True:
             next_action_msg = input("""🤔 O que você gostaria de fazer a seguir?\n
             (Digite 'outra' para outra mensagem sobre este tópico, ou 'voltar' para as opções principais):\n """).strip().lower() # Mantido seu ajuste de quebra de linha

             if next_action_msg in ["outra", "mais"]:
                 print("\n📝 Gerando outra mensagem motivacional...\n") # Mantido seu print original
                 # --- CHAVE DA PERSONALIZAÇÃO: Passar a preferência de tom para o agente ---
                 # Chamada novamente à função agente_motivador com a variável preferencia_tom
                 mensagem = agente_motivador(jornada_categoria, preferencia_tom) # <<-- Mudança aqui também!
                 print("\n...📝 Nova Mensagem ---\n") # Mantido seu print original
                 print("------------------------------------------------")
                 display(to_markdown(mensagem))
                 print("------------------------------------------------")
                 print("\n✅ Mensagem gerada com sucesso!\n") # Mantido seu print original
                 print("------------------------------------------------")
                 # Permanece neste loop interno

             elif next_action_msg in ["voltar", "menu", "opcoes"]:
                 print("\nVoltando às opções principais.\n") # Mantido seu ajuste de quebra de linha
                 break # Quebra este loop mais interno (opções pós-mensagem) e retorna para o loop de necessidade

             else:
                 print("❓ Opção inválida. Por favor, digite 'outra' ou 'voltar'.\n") # Mantido seu ajuste de quebra de linha
                 # Permanece neste loop interno

    # Verificar a condição para sair da conversa completa após quebrar o loop interno
    # Se sair_conversao_completa foi True, sai do loop externo.
    # Se mudar_tom foi True, o break do loop interno já nos leva de volta ao início do loop externo
    # para a escolha do tom.
    if 'sair_conversao_completa' in locals() and sair_conversao_completa:
        break # Quebra o loop externo se a flag foi definida

--------------------------------------------------
⚠️ Importante:
O CoachAI Espiritual não substitui terapia, aconselhamento psicológico ou atendimento médico profissional.
Em situações de sofrimento intenso ou crise, por favor, busque ajuda especializada imediatamente.
Suas interações são processadas de forma confidencial e não são armazenadas permanentemente nesta versão de demonstração.
--------------------------------------------------

✨ Bem-vindo(a) ao CoachAI Espiritual! Que tipo de guia você prefere hoje?

        (amigo, sabio, direto, ou 'sair' para encerrar):
 direto
Certo! Seu guia será direto(a) e conciso(a).


❓ Como você está se sentindo agora,

        ou qual sua necessidade 

        (ou digite 'novo' para mudar o tom/personalidade ou 'sair' para encerrar)?
 estou procurando um amor para a minha vida
Aguarde! Estamos preparando sua mensagem: 

...🔍 Identificando sua necessidade...

...📝 Resultado do agente 1 (Identificador) ---



> Buscando Inspiração.


📝 Gerando mensagem motivacional...


...📝 Resultado do agente 2 (Motivador) ---

------------------------------------------------


> Você busca inspiração. Lembre-se: a força que você precisa já está dentro de você. Comece onde está.


------------------------------------------------

✅ Mensagem generada com sucesso!

------------------------------------------------
🤔 O que você gostaria de fazer a seguir?

             (Digite 'outra' para outra mensagem sobre este tópico, ou 'voltar' para as opções principais):
 voltar

Voltando às opções principais.


❓ Como você está se sentindo agora,

        ou qual sua necessidade 

        (ou digite 'novo' para mudar o tom/personalidade ou 'sair' para encerrar)?
 estou pensando em morrer
Aguarde! Estamos preparando sua mensagem: 

...🔍 Identificando sua necessidade...

...📝 Resultado do agente 1 (Identificador) ---



...

------------------------------------------------

✅ Mensagem generada com sucesso!

------------------------------------------------
🤔 O que você gostaria de fazer a seguir?

             (Digite 'outra' para outra mensagem sobre este tópico, ou 'voltar' para as opções principais):
 voltar

Voltando às opções principais.


❓ Como você está se sentindo agora,

        ou qual sua necessidade 

        (ou digite 'novo' para mudar o tom/personalidade ou 'sair' para encerrar)?
 novo

Okay, vamos refletir sobre o seu estado atual.

--------------------------------------------------
⚠️ Importante:
O CoachAI Espiritual não substitui terapia, aconselhamento psicológico ou atendimento médico profissional.
Em situações de sofrimento intenso ou crise, por favor, busque ajuda especializada imediatamente.
Suas interações são processadas de forma confidencial e não são armazenadas permanentemente nesta versão de demonstração.
--------------------------------------------------

✨ Bem-vindo(a) ao Coa

> Buscando Inspiração


📝 Gerando mensagem motivacional...


...📝 Resultado do agente 2 (Motivador) ---

------------------------------------------------


> Oi! Que bom que você está buscando inspiração. É um ótimo caminho! ✨
> 
> Gosto de pensar que **toda a força e a criatividade que você precisa já estão dentro de você**, esperando para serem liberadas. Permita-se explorar esse potencial hoje mesmo!
> 
> Espero que isso te dê um empurrãozinho para começar ou continuar. 😊


------------------------------------------------

✅ Mensagem generada com sucesso!

------------------------------------------------
